#### import necessary packages:

In [7]:
import numpy as np
import sqlite3
import datetime

### Task 1:
#### Construct function, which computes total buy and sell volumes.

In [2]:
# First take a look at the table.

conn = sqlite3.connect('trades.sqlite')
cur = conn.cursor()

rows = cur.execute("SELECT * FROM epex_12_20_12_13").fetchall()

for row in rows:
    print(row)

('trade_1', 5, 20.0, 'buy', 'strategy_1')
('trade_2', 17, 10.0, 'sell', 'strategy_1')
('trade_3', 15, 8.0, 'buy', 'strategy_1')
('trade_4', 13, 35.0, 'sell', 'strategy_2')
('trade_5', 23, 11.0, 'sell', 'strategy_2')
('trade_6', 8, 20.0, 'buy', 'strategy_2')


In [3]:
# I am not sure if it is convention to keep them, but i dropped *args and **kwargs in my functions, 
# since i dont use them in my approach.

def compute_total_buy_volume() -> float:    
    """
    Return buy volumes.
    """
    
    # establish connection with database and define cursor
    conn = sqlite3.connect('trades.sqlite')
    cur = conn.cursor()
    
    # collect quantities from all rows with side='buy'
    buy_quantities = cur.execute("SELECT quantity FROM epex_12_20_12_13 WHERE side = 'buy'").fetchall()

    total_buy_volume = np.sum(buy_quantities)

    return(total_buy_volume)


def compute_total_sell_volume() -> float:
    """
    Return sell volumes.
    """

    # establish connection with database and define cursor
    conn = sqlite3.connect('trades.sqlite') 
    cur = conn.cursor()

    # collect quantities from all rows with side='sell'
    sell_quantities = cur.execute("SELECT quantity FROM epex_12_20_12_13 WHERE side = 'sell'").fetchall()

    total_sell_volume = np.sum(sell_quantities)

    return(total_sell_volume)

In [4]:
print('Total buy volume:', compute_total_buy_volume(), 'MWh')
print('Total sell volume:', compute_total_sell_volume(), 'MWh')

Total buy volume: 28 MWh
Total sell volume: 53 MWh


### Task 2:
#### Write a function that computes the PnL of each strategy.

In [19]:
def compute_pnl(strategy_id: str) -> float:
    """
    Compute PnL of strategy.
    """

    # establish connection with database and define cursor
    conn = sqlite3.connect('trades.sqlite')
    cur = conn.cursor()

    # collect quantity and price from all buys and sells with strategy='strategy_id'
    buys = cur.execute("SELECT quantity, price FROM epex_12_20_12_13 WHERE side = 'buy' AND strategy = '{}'".format(strategy_id)).fetchall()
    sells = cur.execute("SELECT quantity, price FROM epex_12_20_12_13 WHERE side = 'sell' AND strategy = '{}'".format(strategy_id)).fetchall()
    
    # calculate PnL as sum of profits of all sells minus sum of costs of all buys
    pnl = - np.sum([np.prod(buys[i]) for i in range(len(buys))]) + np.sum([np.prod(sells[i]) for i in range(len(sells))])

    return(pnl)

In [20]:
print('strategy 1 PnL:', compute_pnl(strategy_id='strategy_1'), 'EUR')
print('strategy 2 PnL:', compute_pnl(strategy_id='strategy_2'), 'EUR')

strategy 1 PnL: -50.0 EUR
strategy 2 PnL: 548.0 EUR


The above tool gives the PnL, which might or might not be insightful depending on usecase. It is important to note that The volume of buys and sells for each strategy is not equal. For strategy 1 we buy 20MWh and sell 17MWh and for strategy 2 we buy 8 MWh and sell 36 MWh. Therefore a good/bad PnL does not necessarily imply a good/bad trading strategy.

One interesting indicator to further evaluate strategies could be something like the average buy price and the average sell price of each strategy. It is implemented below.

In [5]:
def compute_avg_buysell(strategy_id: str) -> dict:
    """
    Compute average buy and sell prices of strategy.
    """   

    # establish connection with database and define cursor
    conn = sqlite3.connect('trades.sqlite')
    cur = conn.cursor()

    # collect quantity and price from all buys and sells with strategy='strategy_id'
    buys = cur.execute("SELECT quantity, price FROM epex_12_20_12_13 WHERE side = 'buy' AND strategy = '{}'".format(strategy_id)).fetchall()
    sells = cur.execute("SELECT quantity, price FROM epex_12_20_12_13 WHERE side = 'sell' AND strategy = '{}'".format(strategy_id)).fetchall()
    
    # calculate total cost for buys and total volume of buys
    total_cost_for_buys = np.sum([np.prod(buys[i]) for i in range(len(buys))])
    total_buy_volume = np.sum([buys[i][0] for i in range(len(buys))])

    avg_buy_price = total_cost_for_buys / total_buy_volume

    # calculate total cost for sells and total volume of sells
    total_profit_from_sells = np.sum([np.prod(sells[i]) for i in range(len(sells))])
    total_sell_volume = np.sum([sells[i][0] for i in range(len(sells))])

    avg_sell_price = total_profit_from_sells / total_sell_volume

    return {
        'strategy' : strategy_id,
        'avg buy' : avg_buy_price, 
        'avg sell' : avg_sell_price
        }

In [6]:
print(compute_avg_buysell(strategy_id='strategy_1'))
print(compute_avg_buysell(strategy_id='strategy_2'))

{'strategy': 'strategy_1', 'avg buy': 11.0, 'avg sell': 10.0}
{'strategy': 'strategy_2', 'avg buy': 20.0, 'avg sell': 19.666666666666668}


### Task 3:
#### Create web app entry point according to given API definition:

In [8]:
from flask import Flask, jsonify

app = Flask(__name__)

# Create API endpoints
@app.route('/v1/pnl/<strategy_id>', methods=['GET'])
def compute_pnl(strategy_id: str):
    
    conn = sqlite3.connect('trades.sqlite')
    cur = conn.cursor()
    
    # get quantity and price of all buys/sells where strategy='strategy_id'
    buys = cur.execute("SELECT quantity, price FROM 'epex_12_20_12_13' WHERE side = 'buy' AND strategy = '{}'".format(strategy_id)).fetchall()
    sells = cur.execute("SELECT quantity, price FROM 'epex_12_20_12_13' WHERE side = 'sell' AND strategy = '{}'".format( strategy_id)).fetchall()
    
    # calculate pnl
    pnl = - np.sum([np.prod(buys[i]) for i in range(len(buys))]) + np.sum([np.prod(sells[i]) for i in range(len(sells))])

    # get the present time as string according to definition
    tick = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")

    # construct response dict
    response_data = {
        "strategy": strategy_id,
        "value": pnl,
        "unit": "euro",
        "capture_time": tick
    }

    # transform dictionary into json response according to definition.
    return jsonify(response_data), 200

This was my first time setting up an API so i am not sure if I did exactly what was asked in the task.

However, if the above code is run as a Python script on a test server, it gives the responses posted below, which should be correct. 

The code regarding the web entry is also uploaded seperately as application.py in the repository.

<img src="strat_1.png" width="800">

<img src="strat_2.png" width="800">

<img src="strat_3.png" width="800">